In [ ]:
import cv2
from ultralytics import YOLO
import easyocr

# -------------------------------
# CONFIGURATION
# -------------------------------

# Path to your trained YOLO door detection model
YOLO_MODEL_PATH = "runs/detect/train/weights/best.pt"

# Initialize YOLO model
door_model = YOLO(YOLO_MODEL_PATH)

# Initialize EasyOCR for text detection
reader = easyocr.Reader(['en'])

def analyze_floor_plan(image_path, target_labels):
    """
    Detects door symbols and highlights specific text labels in a floor plan image.

    Args:
        image_path (str): Path to the floor plan image.
        target_labels (list): List of text labels to highlight, e.g. ["STR", "WASH", "HALL"]

    Returns:
        result_img (ndarray): Annotated image with highlighted text labels.
        door_count (int): Total number of detected door symbols.
        detected_labels (list): List of found labels.
    """

    # -------------------------------
    # STEP 1: Load the image
    # -------------------------------
    image = cv2.imread(image_path)

    # -------------------------------
    # STEP 2: Door detection using YOLO
    # -------------------------------
    door_results = door_model.predict(
        source=image_path,
        conf=0.4,
        device="cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu",
        verbose=False
    )

    # Count total doors
    door_count = len(door_results[0].boxes)

    # -------------------------------
    # STEP 3: Text detection using EasyOCR
    # -------------------------------
    results = reader.readtext(image)

    detected_labels = []

    for (bbox, text, prob) in results:
        text_upper = text.upper().strip()
        if text_upper in [label.upper() for label in target_labels]:
            detected_labels.append(text_upper)

            # Draw rectangle for the detected label
            (top_left, top_right, bottom_right, bottom_left) = bbox
            top_left = tuple(map(int, top_left))
            bottom_right = tuple(map(int, bottom_right))

            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, text_upper, (top_left[0], top_left[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    return image, door_count, detected_labels


# -------------------------------
# USAGE EXAMPLE
# -------------------------------

if __name__ == "__main__":
    # Input floor plan image
    img_path = "dataset/images/val/image51.jpg"

    # Labels to highlight
    target_labels = ["STR"]

    # Run the analysis
    result_img, door_count, detected_labels = analyze_floor_plan(img_path, target_labels)

    # Save the annotated image
    cv2.imwrite("output/annotated_floorplan.jpg", result_img)

    # Show results
    print(f"Total Doors Detected: {door_count}")
    print(f"Detected Labels: {detected_labels}")

    cv2.imshow("Annotated Floor Plan", result_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
